In [171]:
# !pip install -qU "python-gdcm" pydicom pylibjpeg "opencv-python-headless"

In [172]:
!pip install -qU ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl --find-links frozen_packages --no-index

In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
from tqdm.notebook import tqdm
from pathlib import Path
from collections import Counter

import matplotlib as mpl
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle

tqdm.pandas()

In [174]:
# for dcm and nii
import pydicom
import nibabel as nib
from pydicom.pixel_data_handlers.util import apply_voi_lut

# Preparing Main DataFrame


In [175]:
paths = {
    'train_df': Path('../input/rsna-2022-cervical-spine-fracture-detection/train.csv'),
    'train_bbox': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv'),
    'train_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_images'),
    'train_nifti_segments': Path('../input/rsna-2022-cervical-spine-fracture-detection/segmentations'),
    'test_df': Path('../input/rsna-2022-cervical-spine-fracture-detection/test.csv'),
    'test_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/test_images')
}

# **Preparing Main DataFrame**
___

In [176]:
train_df = pd.read_csv(paths['train_df'])
test_df = pd.read_csv(paths['test_df'])

In [177]:
train_df['total_fractures']= train_df.loc[:,[f"C{i}" for i in range(1,8)]].sum(axis=1)

# **Preprocessing**

**Creating Segment Path**

In [178]:
train_df['segment_path'] = train_df['StudyInstanceUID'].map(lambda x: paths['train_images']/x)


**Creating nii segment path**



In [179]:
def add_nii_segment_path(uid):
    base_path = paths['train_nifti_segments']
    # path if exists else None
    path = base_path/(uid+'.nii')
    if path.exists():
        return path
    return None

train_df['nii_segments_path'] = train_df['StudyInstanceUID'].map(add_nii_segment_path)


# train_df['nii_segments_path'] = train_df['StudyInstanceUID'].map(lambda x: paths['train_nifti_segments']+/x)

In [180]:
train_df  #adding segmentpath with studyinstanceUId

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,total_fractures,segment_path,nii_segments_path
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,2,../input/rsna-2022-cervical-spine-fracture-det...,None
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,1,../input/rsna-2022-cervical-spine-fracture-det...,None
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,1,../input/rsna-2022-cervical-spine-fracture-det...,None
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,None
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,1,../input/rsna-2022-cervical-spine-fracture-det...,../input/rsna-2022-cervical-spine-fracture-det...
...,...,...,...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1,3,../input/rsna-2022-cervical-spine-fracture-det...,None
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1,1,../input/rsna-2022-cervical-spine-fracture-det...,None
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,None
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,None


# **Scans in reverse order**

In [181]:
def check_reverse_required(path):
    paths = list(path.glob('*'))
    paths.sort(key=lambda x:int(x.stem))
    z_first = pydicom.dcmread(paths[0]).get("ImagePositionPatient")[-1]
    z_last = pydicom.dcmread(paths[-1]).get("ImagePositionPatient")[-1]
    if z_last < z_first:
        return False
    return True

In [ ]:
checks = train_df['segment_path'].map(check_reverse_required)


In [ ]:
train_df['reverse_required'] = checks
Counter(checks)


In [ ]:
indices_where_reverse_required = [i for i,req in (checks.reset_index()).values if req is True]
print(indices_where_reverse_required)

**DCM Images**

In [ ]:
def get_dcm_images(path):
    paths = list(path.glob('*'))
    paths.sort(key=lambda x:int(x.stem)) # sort based on slice index which is the filename: index.dcm
    data = [pydicom.dcmread(f) for f in paths]
    images = [apply_voi_lut(dcm.pixel_array, dcm) for dcm in data]
    return images

**Nii Images**

In [ ]:
def add_nii_segment_path(uid):
    base_path = paths['train_nifti_segments']
    # path if exists else None
    path = base_path/(uid+'.nii')
    if path.exists():
        return path
    return None


def get_nii_segments(path):
    f = nib.load(path)
    segmentations = f.get_fdata()[:, ::-1, ::-1].transpose(2, 1, 0)
    return segmentations

In [ ]:
sample_index = train_df.iloc[99,:]
dcm_images = get_dcm_images(sample_index['segment_path'])
if sample_index['reverse_required'] == True:
    dcm_images.sort(reverse=True)

segments = get_nii_segments(sample_index['nii_segments_path'])



In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

fig, [ax1,ax2] = plt.subplots(1,2)
ax1.axis('off')
ax2.axis('off')
images = []
for i in tqdm(range(len(dcm_images))):
    im1 = ax1.imshow(dcm_images[i], animated=True, cmap='bone')
    im2 = ax2.imshow(segments[i,:,:], animated=True, cmap='bone')
    if i==0:
        ax1.imshow(dcm_images[i], cmap='bone')
        ax2.imshow(segments[i,:,:], cmap='bone')
    images.append([im1,im2])
        

ani = animation.ArtistAnimation(fig, images, interval=50, blit=True,
                                repeat_delay=1000)
plt.close()
ani